# Add attributes to NLIW

Attrs:
1) Theoretical wave speed (+UQ)
2) Direction (total and near-bed) (nearbed now done in save fluxes)
3) Barotropic velocity (u,v,t), and nearbed IT and BC'
4) Mode-1 Wave Amplitude (time series)

In [1]:
import os
import glob
import numpy as np
import xarray as xr
import arviz as az
from afloat.pca import PCA_2D
from scipy.signal import find_peaks
from afloat.currents import sd_to_uv, uv_to_sd

import sys
sys.path.insert(0, '../')
from chp3_basefuncs import *
from chp3_fluxfuncs import *
from chp3_samplefuncs import sample_density, sample_celerity
from wootils.linalg import polar_stddev, polar_mean

## Loop through and load the events

In [2]:
moor_file = 'T150'
field_trip = 'RS2019'
wave_type = 'NLIW'

# Set constants
# varname = 'Temperature'
depthvar = 'z_nom'
zbl = -130.

# Set key data directories and outputs
project_root = r'/mnt/c/Users/00099894/Large_Data/'
work_dir = r'/mnt/c/Users/00099894/OneDrive - The University of Western Australia/UWA/PhD/pl'
load_dir = os.path.join(project_root, field_trip, wave_type)

wav_tx = get_nliw_times()

In [3]:
# Load the theoretical wave speed (for whole deployment)
trace_name = os.path.join('../', '1_pyddcurves', 'RS2019_T150_3days')
nsamp = 100
mode=0

rhoall, zout, time = sample_density(trace_name, gridpoints=None, samples=nsamp, noise=True)
cel = sample_celerity(time, zout, rhoall, nsamp, mode=mode)

In [4]:
# Load + decompose the ADCP velocities
tree_in = glob.glob(os.path.join(project_root, field_trip, f'*{moor_file}*RDI 150 kHz*.nc'))
ds_adcp = xr.open_dataset(tree_in[0])
z_nom = ds_adcp[depthvar].values + ds_adcp['distance'].values
ds_adcp['distance'] = z_nom

# Decompose function
u_tide, u_itide, u_drift, u_bchf = decompose_xarray(ds_adcp['u'], zbl, t_dim='time', z_dim='distance')
v_tide, v_itide, v_drift, v_bchf = decompose_xarray(ds_adcp['v'], zbl, t_dim='time', z_dim='distance')

In [5]:
# Load the buoyancy mode file
bmode_in = glob.glob(os.path.join(project_root, field_trip, 'pimos_v1.1.dev2', 'FV03', 'bmodes_noknockdown', '*.nc'))
ds_bmode = xr.open_dataset(bmode_in[0])

In [11]:
sigskp = 4*60

# Set the height to extract ADCP velocities at
adcp_h = zbl

# Loop thorugh files
for ii, wt in enumerate(wav_tx): 
    # if ii==6:

        load_path = os.path.join(load_dir, f'{wave_type}_E{int(ii+1)}_flux.nc')
        ds = xr.open_dataset(load_path)
        ds.close()

        # Load the trace
        try:
            trace_name = glob.glob(os.path.join(load_dir, f'Event_{int(ii)}_trace.nc'))[0]
            ds_trace = az.from_netcdf(trace_name)
            good_load = True
        except:
            good_load = False

        # Add the M-1 amplitude
        txx = (ds_bmode['time'] > ds['time'].values[0]) & (ds_bmode['time'] <= ds['time'].values[-1])
        ds['mode_1'] = ds_bmode['A_n'].isel(modes=0, time=txx).interp(time=ds['time'])

        # Add the wave speed
        ix = np.argmin(np.abs(time - np.mean(ds['time']).values))
        ds.attrs['m1_celerity'] = [cel[ix,:].mean(), np.percentile(cel[ix,:], 2.5), np.percentile(cel[ix,:], 97.5)]

        # Add the wave direction
        ds.attrs['wave_direction'] = adcp_theta(u_bchf, v_bchf, ds)[0]
        # ds.attrs['sig_direction'] = sig_theta(ds, skp=4*60)[0]

        # Add the ADCP velocities
        ds['e_adcp'] = ds_adcp['u'].sel(distance=adcp_h, method='nearest').interp(time=ds['time'])
        ds['n_adcp'] = ds_adcp['v'].sel(distance=adcp_h, method='nearest').interp(time=ds['time'])

        ds['e_tide'] = u_tide.interp(time=ds['time'])
        ds['n_tide'] = v_tide.interp(time=ds['time'])
        ds['e_itide'] = u_itide.sel(distance=adcp_h, method='nearest').interp(time=ds['time'])
        ds['n_itide'] = v_itide.sel(distance=adcp_h, method='nearest').interp(time=ds['time'])
        ds['e_bchf'] = u_bchf.sel(distance=adcp_h, method='nearest').interp(time=ds['time'])
        ds['n_bchf'] = v_bchf.sel(distance=adcp_h, method='nearest').interp(time=ds['time'])

        # Add the instrument heading, pitch, roll
        load_enu = os.path.join(load_dir, f'{wave_type}_{int(ii+1)}_ENU.nc')
        ds_enu = xr.open_dataset(load_enu)
        ds_enu.close()
        ds.attrs['heading'] = np.median(ds_enu['heading'].values)
        ds.attrs['pitch'] = np.median(ds_enu['pitch'].values)
        ds.attrs['roll'] = np.median(ds_enu['roll'].values)

        hx = np.arange(len(ds_enu.height.values.copy())-3)
        ds_adj = ds_enu.isel(height=hx)
        _, theta = rotate_data(ds_adj)
        ds.attrs['en_2_uv'] = theta

        # Get the rhat and if good, add speed and dir to dataset
        if good_load:
            r_hat = az.rhat(ds_trace.posterior)
            if r_hat.to_array().values.mean() < 1.2:
                ds.attrs['rhat'] = r_hat.to_array().values.mean()
                ds.attrs['obs_celerity'] = [ds_trace.posterior['c'].mean().values, ds_trace.posterior['c'].std().values]
                ds.attrs['theta'] = [polar_mean(ds_trace.posterior['theta'].values.flatten()),\
                                     polar_stddev(ds_trace.posterior['theta'].values.flatten())]  

                m1, nan_lx, nan_ex = fl.trim_edge_nans_xr(ds['mode_1'])
                px, pp = find_peaks(-ds['mode_1'][nan_lx:nan_ex].values, height=15, distance=100)
                ix_min = px[0]   #np.argmin(m1_hp.values)

                u_obs, v_obs = sd_to_uv(ds.attrs['obs_celerity'][0],\
                                        ds.attrs['theta'][0] + np.deg2rad(ds.attrs['heading']),\
                                        convention='ocean')
                u_ix = ds['e_tide'][nan_lx:nan_ex][ix_min - 5:ix_min + 5].mean(dim='time').values
                v_ix = ds['n_tide'][nan_lx:nan_ex][ix_min - 5:ix_min + 5].mean(dim='time').values
                spd_est, dir_est = uv_to_sd(u_obs - u_ix, v_obs - v_ix, convention='ocean')

                ds.attrs['est_celerity'] = spd_est
                ds.attrs['est_theta'] = dir_est
        
        # Save the cleaned up ds
        ds = ds.drop(['lat_nom', 'lon_nom', 'z_nom', 'beam', 'cartesian_axes', 'distance', 'modes'])
        save_path = os.path.join(load_dir, f'{wave_type}_E{int(ii+1)}_fluxattrs.nc')
        ds.to_netcdf(save_path, mode='w')
        ds.close()
        print(f'Saved {ii+1} of {len(wav_tx)}')


        # # Remove the existing file
        # os.remove(load_path)
        # print(f'Removed {ii+1} of {len(wav_tx)}')

Saved 1 of 11
Saved 2 of 11
Saved 3 of 11
Saved 4 of 11
Saved 5 of 11
Saved 6 of 11
Saved 7 of 11
Saved 8 of 11
Saved 9 of 11
Saved 10 of 11
Saved 11 of 11


In [7]:
ds

<xarray.Dataset>
Dimensions:         (time: 5040, height: 126)
Coordinates:
  * time            (time) datetime64[ns] 2019-03-15T22:30:00.089090 ... 2019...
  * height          (height) float64 0.1 0.16 0.22 0.28 ... 7.42 7.48 7.54 7.6
    turb_SSC        (height, time) float64 ...
    prof_z_nom      float64 -130.1
Data variables: (12/16)
    w2_turb         (height, time) float32 ...
    wc_turb         (height, time) float64 ...
    u_mean          (height, time) float32 ...
    v_mean          (height, time) float32 ...
    w_mean          (height, time) float32 ...
    c_mean          (height, time) float64 ...
    ...              ...
    e_tide          (time) float64 -0.0774 -0.07737 ... -0.0008025 -0.0007827
    n_tide          (time) float64 0.08909 0.08912 ... -0.004629 -0.004663
    e_itide         (time) float64 0.08731 0.08723 0.08714 ... -0.08909 -0.08908
    n_itide         (time) float64 0.0001201 0.0001154 ... 0.02911 0.02909
    e_bchf          (time) float64 -0.04708 -0.04315 ... 0.03317 0.03373
    n_bchf          (time) float64 0.03086 0.02574 0.02063 ... 0.04264 0.03891
Attributes:
    sig_direction:   0.9541637578804518
    m1_celerity:     [0.7663179474224424, 0.751738114380503, 0.7808475115233648]
    wave_direction:  -0.7243300497017439
    heading:         58.48
    pitch:           1.26
    roll:            -3.24
    en_2_uv:         0.9541637578804518
    rhat:            1.0005624410785157
    obs_celerity:    [array(0.74911222), array(0.11217456)]
    theta:           [2.500963614939559, 0.14935700297041765]
    est_celerity:    0.8120158654498805
    est_theta:       -2.8943953714090522